In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [3]:
df = pd.read_csv('indian_languages.csv')

In [4]:
df.head()

,text,language
0,???? ?? ????? ??? ???,Hindi
1,????????? ??????? ???????.,Tamil
2,??? ???????? ???? ??????,Bengali
3,???? ??? ??? ???.,Marathi
4,????? ?????????.,Malayalam
